In [1]:
import geopandas as gpd
import pandas as pd
import json
import shapely
from shapely import wkt

import folium
from folium import plugins

In [28]:
## Access json matching bursts to frame IDs
import requests
from io import BytesIO
import zipfile

# URL of the ZIP file containing the JSON file
repo_zip_url = 'https://github.com/opera-adt/burst_db/releases/download/v0.3.1/frame-geometries-simple-0.3.1.geojson.zip'

# Download the ZIP file
response = requests.get(repo_zip_url)
zip_data = BytesIO(response.content)

# Extract the JSON file from the ZIP archive
with zipfile.ZipFile(zip_data, 'r') as zip_ref:
    # Assuming your JSON file is named 'data.json' within the ZIP
    json_data = zip_ref.read('frame-geometries-simple-0.3.1.geojson')

# Load the JSON data
data = json.loads(json_data.decode('utf-8')) # ['features']

In [29]:
gdf = gpd.GeoDataFrame.from_features(data)
gdf['frameID'] = pd.json_normalize(data["features"])["id"].values     # ID in 'feature' of geojson is added

In [30]:
gdf.head()

,geometry,is_land,is_north_america,orbit_pass,frameID
0,"POLYGON ((2.53190 4.75701, 2.21493 6.28652, 4....",1,False,ASCENDING,4
1,"POLYGON ((2.22069 6.25855, 1.93907 7.62125, 4....",1,False,ASCENDING,5
2,"POLYGON ((1.94483 7.59326, 1.66390 8.95599, 3....",1,False,ASCENDING,6
3,"POLYGON ((1.66965 8.92799, 1.38921 10.29072, 3...",1,False,ASCENDING,7
4,"POLYGON ((1.39093 10.28236, 1.11482 11.62540, ...",1,False,ASCENDING,8


In [31]:
orbitPass = 'DESCENDING'    # 'ASCENDING'
gdf = gdf[gdf['orbit_pass'] == orbitPass].reset_index(drop=True)

poly = 'POLYGON((-156.0342 20.1513,-155.7076 18.9142,-155.1486 18.9557,-155.5192 20.3045,-156.0342 20.1513))'   # can be drawn in ASF Vertex (https://search.asf.alaska.edu)

frameIDs = []
geoms = []

for row in gdf.itertuples():
    _geom = row.geometry
    check_intersects = _geom.intersects(wkt.loads(poly))

    if check_intersects:
        frameIDs.append(row.frameID)
        geoms.append(row.geometry)

center_long, center_lat = shapely.unary_union(geoms).centroid.coords[0]   #centroid of found polygons (for plotting)

print(frameIDs)
print(geoms)

[23210, 23211]
[<POLYGON ((-155.735 19.642, -156.889 19.777, -156.592 21.312, -154.185 21, -...>, <POLYGON ((-155.569 18.232, -157.146 18.446, -156.849 19.981, -154.463 19.66...>]


In [33]:
def getbasemaps():
    # Add custom base maps to folium
    basemaps = {
        'Google Satellite Hybrid': folium.TileLayer(
            tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
            attr = 'Google',
            name = 'Google Satellite',
            overlay = True,
            control = True,
            # opacity = 0.8,
            show = False
        ),
        'Esri Satellite': folium.TileLayer(
            tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
            attr = 'Esri',
            name = 'Esri Satellite',
            overlay = True,
            control = True,
            #opacity = 0.8,
            show = False
        )
    }
    return basemaps

In [34]:
# Interactive map
m = folium.Map(location=[center_lat, center_long], zoom_start=7, tiles="CartoDB positron", show=True)

# Add custom basemaps
basemaps = getbasemaps()
for basemap in basemaps:
    basemaps[basemap].add_to(m)

for _, frameID in zip(geoms,frameIDs):
    geo_j =  gpd.GeoSeries(_).to_json()     #reading to geojson
    geo_j = folium.GeoJson(data=geo_j, style_function=lambda x: {"color": "red"}, control=True,
                           name=frameID,show=True)    #converging to readables in folium 
    geo_j.add_to(m)
    popup = folium.Popup(f'frameID: {frameID}')
    popup.add_to(geo_j)

# layer Control
m.add_child(folium.LayerControl())

#Add fullscreen button
plugins.Fullscreen().add_to(m)

#Add inset minimap image
minimap = plugins.MiniMap(width=300, height=300)
m.add_child(minimap)

#Mouse Position
fmtr = "function(num) {return L.Util.formatNum(num, 3) + ' º ';};"
plugins.MousePosition(position='bottomright', separator=' | ', prefix="Lat/Lon:",
                     lat_formatter=fmtr, lng_formatter=fmtr).add_to(m)

# Display
m

